
# Example Implementation with Vertex AI

In [ ]:
#Install Required Libraries
!pip install google-cloud-aiplatform transformers


In [ ]:
import google.cloud.aiplatform as aiplatform
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Set up your project and location (ensure that your environment is authenticated)
PROJECT_ID = 'lawgpt-423703'
REGION = 'us-central1'  # or another region you are using

# Initialize the Vertex AI client
aiplatform.init(project=PROJECT_ID, location=REGION)

# Load pre-trained BERT model and tokenizer from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize input text
input_text = "Vertex AI makes deploying models easy."
inputs = tokenizer(input_text, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get prediction
logits = outputs.logits
predicted_class = torch.argmax(logits).item()

print(f"Predicted class: {predicted_class}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 0


# Example Storing and Querying Data with Google Cloud Storage and BigQuery


In [ ]:
#Set up GCS and BigQuery authentication
from google.colab import auth
auth.authenticate_user()

# Install Google Cloud libraries if needed
!pip install --upgrade google-cloud-storage google-cloud-bigquery


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


In [ ]:
# Load data into GCS and query using bigquery
from google.cloud import bigquery

PROJECT_ID = 'lawgpt-423703'
GCS_BUCKET = 'my-legal-data-bucket'

# Initialize the BigQuery client
bigquery_client = bigquery.Client(project=PROJECT_ID)

# Define the dataset and table IDs
dataset_id = 'sample_dataset'  # Make sure to change to your preferred dataset name
table_id = 'sample_data'  # This is your table name

# Create the dataset if it doesn't exist
dataset_ref = bigquery_client.dataset(dataset_id)

try:
    # Try to get the dataset, if it exists
    dataset = bigquery_client.get_dataset(dataset_ref)
    print(f"Dataset {dataset_id} already exists.")
except:
    # Create the dataset if it doesn't exist
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"  # Set location, you can change this
    dataset = bigquery_client.create_dataset(dataset)
    print(f"Created dataset {dataset_id}.")

# Create a table schema for your data
schema = [
    bigquery.SchemaField("id", "INTEGER"),
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
]

# Set the table reference
table_ref = bigquery_client.dataset(dataset_id).table(table_id)

# Configure the load job
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip header row in CSV
    autodetect=False
)

# Set the GCS URI for your CSV file
gcs_uri = f'gs://{GCS_BUCKET}/sample_data.csv'

# Load the CSV file from GCS into BigQuery
load_job = bigquery_client.load_table_from_uri(gcs_uri, table_ref, job_config=job_config)

# Wait for the load job to complete
load_job.result()

print(f"Loaded {load_job.output_rows} rows into {dataset_id}.{table_id}")


Dataset sample_dataset already exists.
Loaded 3 rows into sample_dataset.sample_data


#Example Setting Up a Scheduled Backup with Cloud SQL

In [ ]:
import google.auth
from google.cloud import sql_v1
# Authenticate and create a client
credentials, project = google.auth.default()
client = sql_v1.SqlBackupRunsServiceClient(credentials=credentials)
PROJECT_ID = 'YOUR_PROJECT_ID'
INSTANCE_ID = 'MY_INSTANCE_ID'
# Define the backup configuration
backup_config = {
    "settings": {
        "backupConfiguration": {
            "binaryLogEnabled": True,
            "enabled": True,
            "startTime": "03:00"
        }
    }
}
# Apply the backup configuration to the instance
instance_name = "projects/PROJECT_ID/instances/INSTANCE_ID"
client.update_backup_run(instance=instance_name, backup_run=backup_config)


#Example: Enabling Encryption with Customer-Managed Keys

In [ ]:
from google.cloud import storage
# Initialize the client
client = storage.Client()
# Define the bucket and key
bucket_name = 'my-secure-bucket'
kms_key_name= 'projects/my-project/locations/global/keyRings/my-keyring/cryptoKeys/my-key'

# Create the bucket with encryption enabled
bucket = client.bucket(bucket_name)
bucket.encryption = {'defaultKmsKeyName': kms_key_name}
bucket.create()
print(f"Bucket {bucket.name} created with encryption.")

#Example Setting Up Pub/Sub for Real-time Data Synchronization

In [ ]:
from google.cloud import pubsub_v1

PROJECT_ID = 'lawgpt-423703'

# Initialize clients and names
publisher = pubsub_v1.PublisherClient()
subscriber = pubsub_v1.SubscriberClient()
topic_name = f'projects/{PROJECT_ID}/topics/learning-updates'
subscription_name = f'projects/{PROJECT_ID}/subscriptions/learning-sub'

# Create topic and subscription
publisher.create_topic(name=topic_name)
subscriber.create_subscription(name=subscription_name, topic=topic_name)

# Publish message
publisher.publish(topic_name, b'New learning data available')

# Subscribe to messages
def callback(msg):
    print(f"Received: {msg.data}")
    msg.ack()

subscriber.subscribe(subscription_name, callback=callback)
print("Listening for messages...")

Listening for messages...


#Example: Deploying a Model with TensorFlow Lite

In [ ]:
import tensorflow as tf
# Convert a TensorFlow model to TensorFlow Lite
model = tf.keras.models.load_model('learning_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the converted model
with open('learning_model.tflite', 'wb') as f:
    f.write(tflite_model)

#Example: Applying Quantization with TensorFlow Lite

In [ ]:
import tensorflow as tf
# Convert a TensorFlow model to a quantized TensorFlow Lite model
model = tf.keras.models.load_model('learning_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()
# Save the quantized model
with open('learning_model_quantized.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

#Example: Automating Model Retraining with Vertex AI Pipelines

In [ ]:
from kfp.v2 import dsl
from google.cloud import aiplatform
@dsl.pipeline(name="retrain-learning-model-pipeline")
def pipeline():
    # Define the retraining and deployment steps
    training_job = aiplatform.CustomJob(
        display_name="model-training-job",
        script_path="train_model.py",
        container_uri="gcr.io/my-project/training-container",
        model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-1:latest"
    )
    training_job.run()
# Compile and run the pipeline
pipeline.compile(pipeline_func=pipeline, package_path='retrain_pipeline.json')
aiplatform.PipelineJob(display_name="retrain-pipeline-job", template_path='retrain_pipeline.json').run()

#Example: Predicting Learner Performance with Vertex AI

In [ ]:
from google.cloud import aiplatform
# Initialize the AI Platform client
aiplatform.init(project='my-project', location='us-central1')
# Define and train a custom model
model = aiplatform.CustomJob.from_local_script(
    display_name='learner-performance-prediction',
    script_path='train_model.py',
    container_uri='gcr.io/my-project/training-container',
   model_serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-1:latest'
)
model.run()